In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from mpl_toolkits.mplot3d import Axes3D

from typing import List
import glob
import os
import gzip
from tqdm import tqdm
import seaborn as sns
import natsort
import hashlib
import random


In [2]:
def calc_proximity(df, proximity_threshold):
    df = df.dropna()
    x_rf, y_rf, z_rf = pd.to_numeric(df['xs_rf']), pd.to_numeric(df['ys_rf']), pd.to_numeric(df['zs_rf'])
    x_vf, y_vf, z_vf = pd.to_numeric(df['osg_fish1_x']), pd.to_numeric(df['osg_fish1_y']), pd.to_numeric(df['osg_fish1_z'])

    df.loc[:, 'x_rf'] = x_rf
    df.loc[:, 'y_rf'] = y_rf
    df.loc[:, 'z_rf'] = z_rf
    df.loc[:, 'x_vf'] = x_vf
    df.loc[:, 'y_vf'] = y_vf
    df.loc[:, 'z_vf'] = z_vf

    df['euclidean_distance'] = np.sqrt((x_rf - x_vf) ** 2 + (y_rf - y_vf) ** 2 + (z_rf - z_vf) ** 2)
    df['proximity'] = df['euclidean_distance'] < proximity_threshold

    # print(f'proximity first 10 rows: {df["proximity"].head(10)}')

    return df

In [3]:
def calc_relative_heading(df, angular_threshold_degrees, proximity_threshold):

    df = calc_proximity(df, proximity_threshold)

    df = df.dropna()
    df['dxs_rf'] = df['x_rf'].diff()
    df['dys_rf'] = df['y_rf'].diff()
    df['dxs_vf'] = df['x_vf'].diff()
    df['dys_vf'] = df['y_vf'].diff()

    df['heading_rf'] = np.arctan2(df['dys_rf'], df['dxs_rf'])
    df['heading_vf'] = np.arctan2(df['dys_vf'], df['dxs_vf'])

    df['heading_rf'].fillna(0, inplace=True)
    df['heading_vf'].fillna(0, inplace=True)

    heading_rf = df['heading_rf']
    heading_vf = df['heading_vf']

    unwrap_heading_rf = np.unwrap(np.array(heading_rf))
    unwrap_heading_vf = np.unwrap(np.array(heading_vf))

    relative_angle = unwrap_heading_rf - unwrap_heading_vf
    relative_angle_complement = np.mod(relative_angle + np.pi, 2 * np.pi) - np.pi

    concat_relative_angles = np.vstack((relative_angle, relative_angle_complement))
    concat_abs_relative_angles = np.abs(concat_relative_angles)

    min_indices_relative = np.argmin(concat_abs_relative_angles, axis=0)
    relative_heading = concat_relative_angles[min_indices_relative, np.arange(concat_relative_angles.shape[1])]

    df['relative_heading'] = relative_heading

    angular_threshold_radians = np.radians(angular_threshold_degrees)

    angular_threshold_radians = np.radians(angular_threshold_degrees)
    following = np.abs(relative_heading) <= angular_threshold_radians

    # print(f'following first 10 rows: {following[:10]}')
    df['following'] = following
    percent_following = np.mean(following) * 100
    print(f'Percent following: {percent_following:.2f}%')
    
    return df

In [4]:
def coordinate_transition(df, heading_vf):
    x_in, y_in = df['x_rf'] - df['x_vf'], df['y_rf'] - df['y_vf']
    x_o = x_in * np.cos(heading_vf) + y_in * np.sin(heading_vf)
    y_o = -x_in * np.sin(heading_vf) + y_in * np.cos(heading_vf)

    df['x_o'] = x_o
    df['y_o'] = y_o

    # print(f'x_o: {x_o[:10]}')
    # print(f'y_o: {y_o[:10]}')

    return df

In [5]:
def rf_behind_vf(df, angular_threshold_degrees, proximity_threshold, x_limit=0, y_limit=0, ):

    df = calc_relative_heading(df, angular_threshold_degrees, proximity_threshold)

    vf_heading = df['heading_vf'].values

    df = coordinate_transition(df, vf_heading)
    df['rf_behind_vf'] = (df['x_o'] < x_limit) & (df['y_o'] < y_limit)
    rf_behind_vf = df['rf_behind_vf'].values
    # print(f'first 10 rows of rf_behind_vf: {rf_behind_vf[:10]}')
    print(f'proportion of time fish is behind: {np.mean(rf_behind_vf)}')

    return df

In [6]:
def frames30_following(df, streak_length_threshold):
    # Use the boolean values directly
    all_conditions = df['following'] & df['proximity'] & df['rf_behind_vf']

    streak_lengths = []
    counter = 0
    for value in all_conditions:
        if value:
            counter += 1
        else:
            if counter >= streak_length_threshold:
                streak_lengths.append(counter)
            counter = 0
    if counter >= streak_length_threshold:
        streak_lengths.append(counter)

    streaks_over_threshold = np.array(streak_lengths)

    print(f'streaks over threshold: {streaks_over_threshold}')
    print(f'number of streaks over threshold: {len(streaks_over_threshold)}')

    return streaks_over_threshold 


In [7]:
def frames30_following_streaks(df, streak_length_threshold):
    all_conditions = df['following'] & df['proximity'] & df['rf_behind_vf']

    streak_id = 0
    current_streak_id = None
    streak_lengths = {}
    streak_ids = []
    streak_id_sequences = {}
    counter = 0

    for value in all_conditions:
        if value:
            counter += 1
            if current_streak_id is None:
                streak_id += 1
                current_streak_id = streak_id
                streak_id_sequences[current_streak_id] = []
            streak_id_sequences[current_streak_id].append(streak_id)
        else:
            if counter >= streak_length_threshold:
                streak_lengths[current_streak_id] = counter
            counter = 0
            current_streak_id = None
        streak_ids.append(current_streak_id)

    if counter >= streak_length_threshold:
        streak_lengths[current_streak_id] = counter

    df['streak_id'] = streak_ids

    # Map streak lengths to each row in the DataFrame
    df['streak_length'] = df['streak_id'].map(streak_lengths).fillna(0)

    # Map streak ID sequences to each row in the DataFrame
    df['streak_id_sequence'] = df['streak_id'].map(streak_id_sequences).fillna('[]')

    return df



In [8]:
def cartesian_to_polar(x, y):
    r = np.sqrt(x**2 + y**2)
    theta = np.arctan2(y, x)
    return r, theta

In [9]:
def plot_data(df):
    # 3D scatter plot of positions
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(df['x_rf'], df['y_rf'], df['z_rf'], c='r', marker='o', label='Real Fish', s=0.2)
    ax.scatter(df['x_vf'], df['y_vf'], df['z_vf'], c='b', marker='^', label='Virtual Fish', s= 0.2)
    ax.legend()
    # plt.show()

    # Histogram of euclidean distances
    fig = plt.figure()
    plt.hist(df['euclidean_distance'], bins=100, label='Euclidean Distance')
    plt.legend()
    plt.show()

    # Plot of proximity over time
    fig = plt.figure()
    plt.plot(df['proximity'], label='Proximity')
    plt.legend()
    plt.show()

    # Plot of relative heading over time
    fig = plt.figure(figsize=(15, 5))
    plt.plot(df['relative_heading'], label='Relative Heading')
    plt.legend()
    plt.show()

    # Plot of following over time (first 1000 data points)
    fig = plt.figure(figsize=(15, 5))
    plt.plot(df['following'][:1000], label='Following')
    plt.legend()
    plt.show()

    # Histogram of relative heading
    fig = plt.figure(figsize=(15, 5))
    plt.hist(df['relative_heading'], bins=100, label='Relative Heading')
    plt.legend()
    plt.show()

    # Histogram of streaks over 30
    streaks_over_30 = frames30_following(df, 30)
    fig = plt.figure(figsize=(15, 5))
    plt.hist(streaks_over_30, bins=100, label='Streaks Over 30')
    plt.xlabel('Streak Length')
    plt.ylabel('Frequency')
    plt.title('Histogram of Streak Lengths Over 30')
    plt.legend()
    plt.show()

In [10]:
def read_files(folder_path):

    all_files = glob.glob(os.path.join(folder_path + "/*.csv"))
    all_files = natsort.natsorted(all_files)

    fish_data_list = []

    for file_path in tqdm(all_files):
        filename = os.path.basename(file_path)
        print(filename)

        fish_data_list.append(file_path)
 
    return fish_data_list


In [11]:
desired_cols = ['framenumber','Stim_Flag', 'Stim_Flag_dir','frame_diff', 'steps',
       'xs_rf', 'ys_rf', 'zs_rf', 'osg_fish1_x', 'osg_fish1_y', 'osg_fish1_z']

folder_path = r"C:\PhD\experiments\fish_experiments\FishSizeKinematics\processed_data"
output_filtered_data_path = r'C:\PhD\experiments\fish_experiments\FishSizeKinematics\processed_filtered_data'
fish_data_list = read_files(folder_path)
# fish_data_list

100%|██████████| 92/92 [00:00<?, ?it/s]

0712vr3_1_112_a28_processed.csv
0712vr3_2_112_a28_processed.csv
0712vr3_3_112_a28_processed.csv
0712vr3_4_112_a28_processed.csv
0712vr4_1_099_a28_processed.csv
0712vr4_2_099_a28_processed.csv
0712vr4_3_099_a28_processed.csv
0712vr4_4_099_a28_processed.csv
0712vr5_1_064_a28_processed.csv
0712vr5_2_064_a28_processed.csv
0712vr5_3_064_a28_processed.csv
0712vr5_4_064_a28_processed.csv
0712vr6_1_084_a28_processed.csv
0712vr6_2_084_a28_processed.csv
0712vr6_3_084_a28_processed.csv
0712vr6_4_084_a28_processed.csv
1012vr3_1_127_a30_processed.csv
1012vr3_2_127_a30_processed.csv
1012vr3_3_127_a30_processed.csv
1012vr3_4_127_a30_processed.csv
1012vr5_1_104_a30_processed.csv
1012vr5_2_104_a30_processed.csv
1012vr5_3_104_a30_processed.csv
1012vr5_4_104_a30_processed.csv
1012vr6_1_112_a30_processed.csv
1012vr6_2_112_a30_processed.csv
1012vr6_3_112_a30_processed.csv
1012vr6_4_112_a30_processed.csv
2711vr3_1_112_a25_processed.csv
2711vr3_2_112_a25_processed.csv
2711vr3_3_112_a25_processed.csv
2711vr3_

In [12]:
# # Define your ranges
# angles = [30, 45, 60, 90]
# proximity_thresholds = [0.01, 0.02, 0.03, 0.04, 0.05]
# streak_length_thresholds = [20, 30, 40]

# results = []

# # Iterate over each file
# for file_path in tqdm(fish_data_list):
#     df = pd.read_csv(file_path, usecols=desired_cols)
    
#     # Iterate over each angle
#     for angle in angles:
#         # Iterate over each proximity threshold
#         for proximity_threshold in proximity_thresholds:
#             # Iterate over each streak length threshold
#             for streak_length_threshold in streak_length_thresholds:
#                 # Calculate rf_behind_vf_df based on the current angle and proximity threshold
#                 rf_behind_vf_df = rf_behind_vf(df, angle, proximity_threshold, x_limit=0, y_limit=0)
                
#                 # Get the streak lengths over the threshold
#                 streaks_over_threshold = frames30_following(rf_behind_vf_df, streak_length_threshold)
                
#                 # Append the results to the list
#                 results.append({
#                     'file_path': file_path,
#                     'angle': angle,
#                     'proximity_threshold': proximity_threshold,
#                     'streak_length_threshold': streak_length_threshold,
#                     'streak_lengths': streaks_over_threshold
#                 })

# # Convert the list of results to a DataFrame
# streaks_df = pd.DataFrame(results)
# # Add a new column 'number_of_streaks' to the DataFrame
# streaks_df['number_of_streaks'] = streaks_df['streak_lengths'].apply(len)
# streaks_df['filename'] = streaks_df['file_path'].apply(lambda x: os.path.basename(x))
# streaks_df.to_csv(r"C:\PhD\experiments\fish_experiments\FishSizeKinematics\streaks_df.csv", index=False)
    

In [13]:
angle = 30
proximity_threshold = 0.02
streak_length_threshold = 30

# Create an empty DataFrame to collect all results
final_results_df = pd.DataFrame()

# Iterate over each file
for file_path in tqdm(fish_data_list):
    df = pd.read_csv(file_path, usecols=desired_cols)
    
    # Calculate rf_behind_vf_df based on the current angle and proximity threshold
    rf_behind_vf_df = rf_behind_vf(df, angle, proximity_threshold, x_limit=0, y_limit=0)
    
    # Get the DataFrame with streaks over the threshold and streak_id column
    streaks_df_with_id = frames30_following_streaks(rf_behind_vf_df, streak_length_threshold)
    
    # Add the file_path to the DataFrame
    streaks_df_with_id['file_path'] = file_path
    
    # Concatenate the current DataFrame to the final results DataFrame
    final_results_df = pd.concat([final_results_df, streaks_df_with_id], ignore_index=True)



  0%|          | 0/92 [00:00<?, ?it/s]C:\Users\kkumari\AppData\Local\Temp\ipykernel_20684\2286220097.py:10: DtypeWarning: Columns (4,5,6,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, usecols=desired_cols)
C:\Users\kkumari\AppData\Local\Temp\ipykernel_20684\133781669.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'x_rf'] = x_rf
C:\Users\kkumari\AppData\Local\Temp\ipykernel_20684\133781669.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

Percent following: 24.23%
proportion of time fish is behind: 0.2709067173579461


  1%|          | 1/92 [00:03<05:39,  3.73s/it]C:\Users\kkumari\AppData\Local\Temp\ipykernel_20684\2286220097.py:10: DtypeWarning: Columns (4,5,6,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, usecols=desired_cols)
C:\Users\kkumari\AppData\Local\Temp\ipykernel_20684\133781669.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'x_rf'] = x_rf
C:\Users\kkumari\AppData\Local\Temp\ipykernel_20684\133781669.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

Percent following: 19.77%
proportion of time fish is behind: 0.2768137313893943


  2%|▏         | 2/92 [00:07<05:12,  3.47s/it]C:\Users\kkumari\AppData\Local\Temp\ipykernel_20684\2286220097.py:10: DtypeWarning: Columns (4,5,6,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, usecols=desired_cols)
C:\Users\kkumari\AppData\Local\Temp\ipykernel_20684\133781669.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'x_rf'] = x_rf
C:\Users\kkumari\AppData\Local\Temp\ipykernel_20684\133781669.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

...

 98%|█████████▊| 90/92 [15:15:37<07:01, 210.68s/it]C:\Users\kkumari\AppData\Local\Temp\ipykernel_20684\2286220097.py:10: DtypeWarning: Columns (4,5,6,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, usecols=desired_cols)
C:\Users\kkumari\AppData\Local\Temp\ipykernel_20684\133781669.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'x_rf'] = x_rf
C:\Users\kkumari\AppData\Local\Temp\ipykernel_20684\133781669.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

Percent following: 19.97%
proportion of time fish is behind: 0.2382672737123144


 99%|█████████▉| 91/92 [15:19:16<03:33, 213.19s/it]C:\Users\kkumari\AppData\Local\Temp\ipykernel_20684\2286220097.py:10: DtypeWarning: Columns (4,5,6,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, usecols=desired_cols)
C:\Users\kkumari\AppData\Local\Temp\ipykernel_20684\133781669.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'x_rf'] = x_rf
C:\Users\kkumari\AppData\Local\Temp\ipykernel_20684\133781669.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

Percent following: 18.66%
proportion of time fish is behind: 0.23602712725086025


100%|██████████| 92/92 [15:23:12<00:00, 602.09s/it]


In [14]:
# Add the 'filename' column to the final results DataFrame
final_results_df['filename'] = final_results_df['file_path'].apply(lambda x: os.path.basename(x))


In [15]:
final_results_df = final_results_df.drop(columns=['file_path', 'osg_fish1_x', 'osg_fish1_y', 'osg_fish1_z', 'dxs_rf', 'dys_rf', 'xs_rf', 'ys_rf', 'zs_rf'])

In [16]:
final_results_df.head()

,framenumber,Stim_Flag,Stim_Flag_dir,frame_diff,steps,x_rf,y_rf,z_rf,x_vf,y_vf,...,heading_vf,relative_heading,following,x_o,y_o,rf_behind_vf,streak_id,streak_length,streak_id_sequence,filename
0,1661.0,6,1,1.0,0.000359,0.060008,-0.067762,-0.052620,0.12,0.0,...,0.0,0.000000,True,-0.059992,-0.067762,True,NaN,0.0,[],0712vr3_1_112_a28_processed.csv
1,1662.0,6,1,1.0,0.000423,0.060182,-0.067406,-0.052649,0.12,0.0,...,0.0,1.116573,False,-0.059818,-0.067406,True,NaN,0.0,[],0712vr3_1_112_a28_processed.csv
2,1663.0,6,1,1.0,0.000389,0.060350,-0.067062,-0.052677,0.12,0.0,...,0.0,1.117008,False,-0.059650,-0.067062,True,NaN,0.0,[],0712vr3_1_112_a28_processed.csv
3,1664.0,6,1,1.0,0.000371,0.060512,-0.066727,-0.052698,0.12,0.0,...,0.0,1.119429,False,-0.059488,-0.066727,True,NaN,0.0,[],0712vr3_1_112_a28_processed.csv
4,1665.0,6,1,1.0,0.000324,0.060668,-0.066404,-0.052712,0.12,0.0,...,0.0,1.122033,False,-0.059332,-0.066404,True,NaN,0.0,[],0712vr3_1_112_a28_processed.csv


In [19]:
final_results_df.to_csv(
    r"C:\PhD\experiments\fish_experiments\FishSizeKinematics\streaks_df_streak_length_id20240208.csv.gz",
    index=False,
    compression='gzip'
)

In [ ]:

# Save the final DataFrame to a CSV file
# final_results_df.to_csv(r"C:\PhD\experiments\fish_experiments\FishSizeKinematics\streaks_df_streak_id20240206.csv", index=False)
